In [14]:
import stim

# Build the Bell pair |Φ+> with H(0); CX(0,1)
s = stim.TableauSimulator()
s.x(0)
s.x(1)


# 1) The state's stabilizer generators (the “bottom n rows” idea)
print("Stabilizers of the current state:")
for e in s.canonical_stabilizers():
    print(repr(e))   # expect +XX and +ZZ on qubits 0–1

# 2) The circuit’s tableau (generator conjugation rules)
# Stim tracks the *inverse* tableau internally; invert it to get the forward tableau.
T_inv = s.current_inverse_tableau()
print("\nForward tableau of the circuit so far:")
print(T_inv**-1)


Stabilizers of the current state:
stim.PauliString("-Z_")
stim.PauliString("-_Z")

Forward tableau of the circuit so far:
+-xz-xz-
| +- +-
| XZ __
| __ XZ


In [3]:
c = stim.Circuit("H 0\nCX 0 1\n")
print(stim.Tableau.from_circuit(c))


+-xz-xz-
| ++ ++
| ZX _Z
| _X XZ


In [2]:
# Requires: pip install stim
import stim
import numpy as np

def sample_bell_in_z(n_shots=50_000):
    """Sample Z-basis outcomes from the Bell state."""
    counts = np.zeros((2, 2), dtype=int)  # [z0, z1]
    for _ in range(n_shots):
        sim = stim.TableauSimulator()
        sim.h(0)          # prepare: H on qubit 0
        sim.cx(0, 1)      # then CNOT 0->1 (CX in Stim API)
        z0 = sim.measure(0)
        z1 = sim.measure(1)
        counts[int(z0), int(z1)] += 1
    return counts

def sample_bell_in_x(n_shots=50_000):
    """Sample X-basis by rotating with H then measuring in Z."""
    counts = np.zeros((2, 2), dtype=int)  # [x0, x1]
    for _ in range(n_shots):
        sim = stim.TableauSimulator()
        sim.h(0)
        sim.cx(0, 1)
        # Measure X by conjugating with H then measuring Z
        sim.h(0); sim.h(1)
        x0 = sim.measure(0)
        x1 = sim.measure(1)
        counts[int(x0), int(x1)] += 1
    return counts

def collapse_demo(n_trials=10):
    """Show that measuring Z on qubit 0 collapses qubit 1 deterministically."""
    results = []
    for _ in range(n_trials):
        sim = stim.TableauSimulator()
        sim.h(0)
        sim.cx(0, 1)

        z0 = sim.measure(0)   # collapse the pair by measuring qubit 0 in Z
        z1 = sim.measure(1)   # now qubit 1 is fixed to the same value
        results.append((int(z0), int(z1)))
    return results

# --- Run it ---
cz = sample_bell_in_z()
cx = sample_bell_in_x()
pairs_after_collapse = collapse_demo(12)

n = cz.sum()
p_z0 = cz.sum(axis=1) / n           # marginal of qubit 0 in Z basis
p_z1 = cz.sum(axis=0) / n           # marginal of qubit 1 in Z basis
p_z_pairs = cz / n                  # joint distribution

m = cx.sum()
p_x0 = cx.sum(axis=1) / m           # marginal of qubit 0 in X basis
p_x1 = cx.sum(axis=0) / m
p_x_pairs = cx / m

print("Z-basis joint distribution P(z0,z1):")
print(p_z_pairs)
print("Z-basis marginals: P(z0)=", p_z0, "  P(z1)=", p_z1)

print("\nX-basis joint distribution P(x0,x1):")
print(p_x_pairs)
print("X-basis marginals: P(x0)=", p_x0, "  P(x1)=", p_x1)

print("\nCollapse demo (measure Z on qubit 0, then Z on qubit 1):")
print(pairs_after_collapse)


Z-basis joint distribution P(z0,z1):
[[0.4984 0.    ]
 [0.     0.5016]]
Z-basis marginals: P(z0)= [0.4984 0.5016]   P(z1)= [0.4984 0.5016]

X-basis joint distribution P(x0,x1):
[[0.50152 0.     ]
 [0.      0.49848]]
X-basis marginals: P(x0)= [0.50152 0.49848]   P(x1)= [0.50152 0.49848]

Collapse demo (measure Z on qubit 0, then Z on qubit 1):
[(0, 0), (0, 0), (0, 0), (0, 0), (1, 1), (1, 1), (0, 0), (0, 0), (1, 1), (0, 0), (1, 1), (1, 1)]


## Testing StabilizerState

In [4]:
import numpy as np
import stim
from sequence.kernel.quantum_state import StabilizerState

def test_stabilizer_state():
    """Test the StabilizerState class functionality including tableau."""
    
    print("=" * 60)
    print("Testing StabilizerState Class with Tableau")
    print("=" * 60)
    
    # Test 1: Single qubit |0⟩ state
    print("\n1. Testing single qubit |0⟩ state...")
    state_0 = StabilizerState(keys=[0], shots=1000)
    print(f"   Keys: {state_0.keys}")
    print(f"   Circuit: {state_0.circuit}")
    print(f"   Density matrix:\n{np.round(state_0.state, 3)}")
    expected = np.array([[1, 0], [0, 0]], dtype=complex)
    print(f"   Expected: |0⟩⟨0| = \n{expected}")
    assert np.allclose(state_0.state, expected, atol=0.1)
    
    # Test tableau
    print(f"   Tableau: {state_0.tableau}")
    assert state_0.tableau is not None
    print(f"   Z sign for qubit 0: {state_0.tableau.z_sign(0)}")
    assert state_0.tableau.z_sign(0) == +1
    print("   ✓ Passed")
    
    # Test 2: Single qubit |1⟩ state
    print("\n2. Testing single qubit |1⟩ state...")
    circuit_1 = stim.Circuit("X 0")
    state_1 = StabilizerState(keys=[0], circuit=circuit_1, shots=1000)
    print(f"   Circuit: {circuit_1}")
    print(f"   Density matrix:\n{np.round(state_1.state, 3)}")
    expected = np.array([[0, 0], [0, 1]], dtype=complex)
    print(f"   Expected: |1⟩⟨1| = \n{expected}")
    assert np.allclose(state_1.state, expected, atol=0.1)
    
    print(f"   Tableau: {state_1.tableau}")
    assert state_1.tableau is not None
    print(f"   Z sign for qubit 0: {state_1.tableau.z_sign(0)}")
    assert state_1.tableau.z_sign(0) == -1
    print("   ✓ Passed")
    
    # Test 3: Single qubit |+⟩ state
    print("\n3. Testing single qubit |+⟩ state...")
    circuit_plus = stim.Circuit("H 0")
    state_plus = StabilizerState(keys=[0], circuit=circuit_plus, shots=10000)
    print(f"   Circuit: {circuit_plus}")
    print(f"   Density matrix:\n{np.round(state_plus.state, 3)}")
    expected = np.array([[0.5, 0.5], [0.5, 0.5]], dtype=complex)
    print(f"   Expected: |+⟩⟨+| = \n{expected}")
    assert np.allclose(state_plus.state, expected, atol=0.05)
    
    print(f"   Tableau: {state_plus.tableau}")
    assert state_plus.tableau is not None
    
    # Verify tableau correctly represents |+⟩
    print(f"   X sign: {state_plus.tableau.x_sign(0)}")
    assert state_plus.tableau.x_sign(0) == +1  # X stabilizes |+⟩
    
    # Use x_output and z_output instead of x_output_pauli
    x_out = state_plus.tableau.x_output(0)
    z_out = state_plus.tableau.z_output(0)
    print(f"   X output: {x_out}")
    print(f"   Z output: {z_out}")
    # After H: X -> Z and Z -> X
    assert str(x_out) == "+Z"
    assert str(z_out) == "+X"
    print("   ✓ Passed")
    
    # Test 4: Bell state on two qubits
    print("\n4. Testing Bell state |Φ+⟩ on qubits 0,1...")
    bell_circuit = stim.Circuit("H 0\nCX 0 1")
    bell_state = StabilizerState(keys=[0, 1], circuit=bell_circuit, shots=10000)
    print(f"   Circuit: {bell_circuit}")
    print(f"   Keys: {bell_state.keys}")
    print(f"   Density matrix shape: {bell_state.state.shape}")
    print(f"   Density matrix:\n{np.round(bell_state.state, 3)}")
    expected = np.array([[0.5, 0, 0, 0.5],
                        [0, 0, 0, 0],
                        [0, 0, 0, 0],
                        [0.5, 0, 0, 0.5]], dtype=complex)
    print(f"   Expected |Φ+⟩⟨Φ+| = \n{expected}")
    assert np.allclose(bell_state.state, expected, atol=0.05)
    
    # Test Bell state tableau
    print(f"   Bell tableau: {bell_state.tableau}")
    assert bell_state.tableau is not None
    print("   ✓ Passed")
    
    # Test 5: Partial trace - Bell state traced to single qubit
    print("\n5. Testing partial trace of Bell state to qubit 0...")
    bell_traced = StabilizerState(keys=[0], circuit=bell_circuit, shots=10000)
    print(f"   Tracing out qubit 1 from Bell state")
    print(f"   Keys after trace: {bell_traced.keys}")
    print(f"   Density matrix:\n{np.round(bell_traced.state, 3)}")
    expected = np.array([[0.5, 0], [0, 0.5]], dtype=complex)
    print(f"   Expected (maximally mixed): I/2 = \n{expected}")
    assert np.allclose(bell_traced.state, expected, atol=0.05)
    
    # Tableau should exist
    print(f"   Tableau still exists: {bell_traced.tableau is not None}")
    assert bell_traced.tableau is not None
    print("   ✓ Passed")
    
    # Test 6: Using set() method and tableau invalidation
    print("\n6. Testing set() method and tableau invalidation...")
    state = StabilizerState(keys=[0])
    
    # Access tableau to force computation
    print(f"   Initial tableau: {state.tableau}")
    tableau_before = str(state.tableau)
    print(f"   Tableau cached: {state._tableau is not None}")
    assert state._tableau is not None
    
    new_circuit = stim.Circuit("H 0\nS 0")  # |Y+⟩ state
    print(f"   Setting new circuit: {new_circuit}")
    state.set(new_circuit)
    
    # After set(), internal tableau should be invalidated
    print(f"   Tableau cleared after set(): {state._tableau is None}")
    assert state._tableau is None
    
    # Access tableau again to recompute
    print(f"   New tableau: {state.tableau}")
    tableau_after = str(state.tableau)
    print(f"   Tableau recomputed: {state._tableau is not None}")
    assert state._tableau is not None
    print(f"   Tableaux different: {tableau_before != tableau_after}")
    assert tableau_before != tableau_after
    print("   ✓ Passed")
    
    # Test 7: Measurement
    print("\n7. Testing measurement...")
    circuit_meas = stim.Circuit("H 0")
    state_meas = StabilizerState(keys=[0], circuit=circuit_meas)
    print(f"   State: |+⟩ (after H gate)")
    
    # Z measurements should be roughly 50/50 for |+⟩
    results_z = [state_meas.measure([0], basis='Z')[0] for _ in range(100)]
    z_ratio = sum(results_z) / len(results_z)
    print(f"   Z-basis measurements (100 samples): {sum(results_z)} ones, {100-sum(results_z)} zeros")
    print(f"   Ratio of 1s: {z_ratio:.2f} (expect ~0.5)")
    assert 0.3 < z_ratio < 0.7
    
    # X measurements should always be 0 for |+⟩
    results_x = [state_meas.measure([0], basis='X')[0] for _ in range(10)]
    print(f"   X-basis measurements (10 samples): {results_x}")
    print(f"   All zeros: {all(r == 0 for r in results_x)}")
    assert all(r == 0 for r in results_x)
    print("   ✓ Passed")
    
    # Test 8: Copy functionality with tableau
    print("\n8. Testing copy (tableau should not be copied)...")
    original = StabilizerState(keys=[0], circuit=stim.Circuit("H 0"))
    
    # Force tableau computation in original
    _ = original.tableau
    print(f"   Original has cached tableau: {original._tableau is not None}")
    assert original._tableau is not None
    
    copied = original.copy()
    print(f"   Original keys: {original.keys}, Copied keys: {copied.keys}")
    assert original.keys == copied.keys
    print(f"   States equal: {np.allclose(original.state, copied.state)}")
    assert np.allclose(original.state, copied.state)
    print(f"   Copied tableau initially None: {copied._tableau is None}")
    assert copied._tableau is None  # Should not be copied
    
    # Access copied tableau to verify it's computed independently
    _ = copied.tableau
    print(f"   Copied tableau after access: {copied._tableau is not None}")
    assert copied._tableau is not None
    print("   ✓ Passed")
    
    # Test 9: Serialization should fail
    print("\n9. Testing serialization (should raise error)...")
    try:
        serialized = state_0.serialize()
        assert False, "Serialization should have raised NotImplementedError"
    except NotImplementedError as e:
        print(f"   Serialize correctly raised: {type(e).__name__}")
    
    try:
        state_0.deserialize({})
        assert False, "Deserialization should have raised NotImplementedError"
    except NotImplementedError as e:
        print(f"   Deserialize correctly raised: {type(e).__name__}")
    print("   ✓ Passed")
    
    # Test 10: GHZ state on 3 qubits with tableau
    print("\n10. Testing GHZ state with tableau...")
    ghz_circuit = stim.Circuit("H 0\nCX 0 1\nCX 0 2")
    ghz_partial = StabilizerState(keys=[0, 1], circuit=ghz_circuit, shots=10000)
    print(f"   GHZ circuit: {ghz_circuit}")
    print(f"   Measuring only qubits: {ghz_partial.keys}")
    print(f"   Density matrix shape: {ghz_partial.state.shape}")
    assert ghz_partial.state.shape == (4, 4)
    
    # Tableau should exist
    print(f"   Tableau exists: {ghz_partial.tableau is not None}")
    assert ghz_partial.tableau is not None
    print(f"   Tableau: {ghz_partial.tableau}")
    print("   ✓ Passed")
    
    # Test 11: Verify truncation is stored
    print("\n11. Testing truncation parameter...")
    state_trunc = StabilizerState(keys=[0], truncation=3)
    print(f"   Truncation value: {state_trunc.truncation}")
    assert state_trunc.truncation == 3
    print("   ✓ Passed")
    
    # Test 12: Tableau lazy initialization
    print("\n12. Testing tableau lazy initialization...")
    lazy_state = StabilizerState(keys=[0, 1])
    print(f"   Tableau initially None: {lazy_state._tableau is None}")
    assert lazy_state._tableau is None
    
    # Access tableau property to trigger computation
    tableau = lazy_state.tableau
    print(f"   Accessed tableau: {tableau}")
    print(f"   Tableau now cached: {lazy_state._tableau is not None}")
    assert lazy_state._tableau is not None
    assert tableau is not None
    print("   ✓ Passed")
    
    # Test 13: Complex circuit tableau
    print("\n13. Testing complex circuit tableau...")
    complex_circuit = stim.Circuit("""
        H 0
        S 0
        CX 0 1
        H 1
        CZ 1 2
    """)
    print(f"   Complex circuit: {complex_circuit}")
    complex_state = StabilizerState(keys=[0, 1, 2], circuit=complex_circuit, shots=10000)
    
    print(f"   Tableau exists: {complex_state.tableau is not None}")
    assert complex_state.tableau is not None
    print(f"   State shape: {complex_state.state.shape}")
    assert complex_state.state.shape == (8, 8)
    print(f"   Tableau: {complex_state.tableau}")
    print("   ✓ Passed")
    
    # Test 14: Test tableau methods
    print("\n14. Testing tableau methods...")
    test_state = StabilizerState(keys=[0], circuit=stim.Circuit("H 0"))
    tableau = test_state.tableau
    
    # Test various tableau methods
    print(f"   Has x_output: {hasattr(tableau, 'x_output')}")
    assert hasattr(tableau, 'x_output')
    print(f"   Has z_output: {hasattr(tableau, 'z_output')}")
    assert hasattr(tableau, 'z_output')
    print(f"   Has to_stabilizers: {hasattr(tableau, 'to_stabilizers')}")
    assert hasattr(tableau, 'to_stabilizers')
    
    # Get stabilizers
    stabilizers = tableau.to_stabilizers()
    print(f"   Stabilizers: {stabilizers}")
    print(f"   Number of stabilizers: {len(stabilizers)}")
    assert len(stabilizers) > 0
    print("   ✓ Passed")
    
    # Test 15: Tableau inverse
    print("\n15. Testing tableau inverse...")
    h_state = StabilizerState(keys=[0], circuit=stim.Circuit("H 0"))
    tableau = h_state.tableau
    print(f"   Original tableau: {tableau}")
    
    inverse = tableau.inverse()
    print(f"   Inverse tableau: {inverse}")
    
    # H is self-inverse, so applying H twice should give identity
    identity = tableau.then(inverse)
    print(f"   Tableau * Inverse: {identity}")
    print(f"   Z sign after H*H^-1: {identity.z_sign(0)}")
    assert identity.z_sign(0) == +1
    print("   ✓ Passed")
    
    # Test 16: Tableau from_circuit static method
    print("\n16. Testing tableau creation methods...")
    circuit = stim.Circuit("H 0\nCX 0 1")
    print(f"   Creating tableau from circuit: {circuit}")
    tableau_from_circuit = stim.Tableau.from_circuit(circuit)
    print(f"   Tableau created: {tableau_from_circuit is not None}")
    assert tableau_from_circuit is not None
    print(f"   Tableau: {tableau_from_circuit}")
    print("   ✓ Passed")
    
    # Test 17: Test x_output_pauli with correct arguments
    print("\n17. Testing x_output_pauli and z_output_pauli...")
    test_tableau = stim.Tableau.from_circuit(stim.Circuit("H 0"))
    # x_output_pauli(input_index, output_index) returns the Pauli at output_index
    # when X is applied to input_index
    x_pauli = test_tableau.x_output_pauli(0, 0)  # X on qubit 0, check qubit 0
    z_pauli = test_tableau.z_output_pauli(0, 0)  # Z on qubit 0, check qubit 0
    print(f"   X output pauli (0,0): {x_pauli} (0=I, 1=X, 2=Y, 3=Z)")
    print(f"   Z output pauli (0,0): {z_pauli}")
    # After H: X->Z (3) and Z->X (1)
    assert x_pauli == 3  # Z
    assert z_pauli == 1  # X
    print("   ✓ Passed")
    
    print("\n" + "=" * 60)
    print("All tests passed successfully!")
    print("=" * 60)

if __name__ == "__main__":
    test_stabilizer_state()

Testing StabilizerState Class with Tableau

1. Testing single qubit |0⟩ state...
   Keys: [0]
   Circuit: 
   Density matrix:
[[ 1.   +0.j    -0.016-0.033j]
 [-0.016+0.033j  0.   +0.j   ]]
   Expected: |0⟩⟨0| = 
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]
   Tableau: +-xz-
| ++
| XZ
   Z sign for qubit 0: 1
   ✓ Passed

2. Testing single qubit |1⟩ state...
   Circuit: X 0
   Density matrix:
[[0.   +0.j    0.007-0.008j]
 [0.007+0.008j 1.   +0.j   ]]
   Expected: |1⟩⟨1| = 
[[0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
   Tableau: +-xz-
| +-
| XZ
   Z sign for qubit 0: -1
   ✓ Passed

3. Testing single qubit |+⟩ state...
   Circuit: H 0
   Density matrix:
[[0.505+0.j    0.5  -0.005j]
 [0.5  +0.005j 0.495+0.j   ]]
   Expected: |+⟩⟨+| = 
[[0.5+0.j 0.5+0.j]
 [0.5+0.j 0.5+0.j]]
   Tableau: +-xz-
| ++
| ZX
   X sign: 1
   X output: +Z
   Z output: +X
   ✓ Passed

4. Testing Bell state |Φ+⟩ on qubits 0,1...
   Circuit: H 0
CX 0 1
   Keys: [0, 1]
   Density matrix shape: (4, 4)
   Density matrix:
[[ 0.496+0.j     0

## Test QuantumManagerStabilizer

In [2]:
"""
QuantumManagerStabilizer with integrated transpiler support for SeQUeNCe compatibility.
"""

import stim
import numpy as np
from typing import List, Optional, Union, Dict, Any
import itertools

# Import the simplified transpiler functions
from transpiler import (
    amplitudes_to_stim_circuit,
    sequence_circuit_to_stim
)

from sequence.kernel.quantum_state import StabilizerState
from sequence.kernel.quantum_manager import QuantumManager
from sequence.constants import STABILIZER_FORMALISM


@QuantumManager.register(STABILIZER_FORMALISM)
class QuantumManagerStabilizer(QuantumManager):
    """
    Quantum manager for stabilizer formalism using Stim backend.
    
    Supports multiple input formats:
    - Stim circuits (direct)
    - Amplitude arrays (transpiled)
    - SeQUeNCe Circuit objects (transpiled)
    
    Key design principles:
    - Each state stores its own Stim circuit and keys
    - States start as single-qubit (2x2 density matrices)
    - Manual grouping combines circuits while preserving qubit indices
    - Efficient Pauli tomography via compiled samplers
    """
    
    def __init__(self, truncation: int = 1, shots: int = 1000, **kwargs):
        """
        Initialize the stabilizer quantum manager.
        
        Args:
            truncation: Dormant parameter for future compatibility
            shots: Number of samples for Pauli tomography (default 1000)
        """
        super().__init__(truncation=truncation)
        self.shots = shots
    
    def new(self, state=None) -> int:
        """
        Create a new qubit with flexible state initialization.
        
        Args:
            state: Can be:
                - None: Creates qubit in |0⟩ state
                - stim.Circuit: Direct circuit that prepares the state
                - List/tuple of complex: Amplitude array to convert
                - Tuple like (complex(1), complex(0)) for SeQUeNCe compatibility
        
        Returns:
            Key for the new qubit
        
        Raises:
            ValueError: If state is not stabilizer or input invalid
        """
        # Convert state to stim.Circuit if needed
        initial_circuit = self._convert_to_stim_circuit(state)
        
        # Create the quantum state key
        key = self._least_available
        self._least_available += 1
        
        # Create state object with the circuit
        state_obj = StabilizerState(
            keys=[key],
            circuit=initial_circuit,
            shots=self.shots,
            truncation=self.truncation
        )
        
        self.states[key] = state_obj
        return key
    
    def run_circuit(self, circuit, keys: List[int], meas_samp=None) -> Dict[int, int]:
        """
        Run a circuit on specified qubits.
        
        Args:
            circuit: Can be:
                - stim.Circuit: Direct Stim circuit
                - SeQUeNCe Circuit object: Will be transpiled
            keys: Keys of qubits that circuit positions map to
            meas_samp: Random sample for measurement (if needed)
        
        Returns:
            Dictionary of measurement results (empty if no measurements)
        
        Raises:
            ValueError: If circuit contains unsupported gates
        """
        # Convert to Stim circuit if needed
        if isinstance(circuit, stim.Circuit):
            stim_circuit = circuit
        elif hasattr(circuit, 'gates') and hasattr(circuit, 'measured_qubits'):
            # SeQUeNCe Circuit object - transpile it
            try:
                stim_circuit = sequence_circuit_to_stim(circuit)
            except ValueError as e:
                raise ValueError(f"Cannot run SeQUeNCe circuit on stabilizer manager: {e}")
        else:
            raise TypeError(
                f"circuit must be stim.Circuit or SeQUeNCe Circuit object, got {type(circuit)}"
            )
        
        # Parse and execute the Stim circuit
        measurements = []
        
        for instruction in stim_circuit:
            gate_name = instruction.name
            targets = [t.value for t in instruction.targets_copy()]
            
            if gate_name == 'M':
                measurements.extend(targets)
            else:
                # Check if multi-qubit gate needs grouping
                if len(targets) > 1:
                    gate_keys = [keys[i] for i in targets if i < len(keys)]
                    states = [self.states[k] for k in gate_keys if k in self.states]
                    if states and not all(s == states[0] for s in states):
                        self.group_qubits(gate_keys)
                
                # Map targets to actual qubit keys
                gate_qubits = []
                for t in targets:
                    if t < len(keys):
                        gate_qubits.append(keys[t])
                    else:
                        gate_qubits.append(t)
                
                if gate_qubits:
                    # Get the state and add operation to its circuit
                    state = self.states[gate_qubits[0]]
                    state.circuit.append(gate_name, gate_qubits)
        
        # Handle measurements
        if measurements:
            results = {}
            measured_keys = [keys[i] for i in measurements if i < len(keys)]
            
            if measured_keys:
                state = self.states[measured_keys[0]]
                
                # Create measurement circuit
                meas_circuit = state.circuit.copy()
                for key in measured_keys:
                    meas_circuit.append("M", [key])
                
                # Sample once
                sampler = meas_circuit.compile_sampler()
                if meas_samp is not None:
                    seed = int(meas_samp * 2**32)
                    sample = sampler.sample(shots=1, seed=seed)[0]
                else:
                    sample = sampler.sample(shots=1)[0]
                
                # Extract results and create new single-qubit states
                for i, key in enumerate(measured_keys):
                    results[key] = int(sample[i]) if i < len(sample) else 0
                    
                    # Create new single-qubit state for measured qubit
                    new_circuit = stim.Circuit()
                    if results[key] == 1:
                        new_circuit.append("X", [key])
                    
                    self.states[key] = StabilizerState(
                        keys=[key],
                        circuit=new_circuit,
                        shots=self.shots,
                        truncation=self.truncation
                    )
                
                return results
        
        # Update density matrices for affected states
        if keys:
            state = self.states[keys[0]]
            state.state = state._compute_density_matrix()
        
        return {}
    
    def set(self, keys: List[int], state=None, preserve_history: bool = False) -> None:
        """
        Set qubits to a state with flexible input support.
        
        Args:
            keys: List of qubit keys to set
            state: Can be:
                - None: Sets to |0...0⟩
                - stim.Circuit: Direct circuit preparation
                - List[complex]: Amplitude array (ket vector)
                - List[List[complex]]: Density matrix (will extract amplitudes if pure)
            preserve_history: If True, append to existing circuit
        
        Raises:
            ValueError: If state not stabilizer or input invalid
        """
        # Convert to Stim circuit
        circuit = self._convert_to_stim_circuit(state, num_qubits=len(keys))
        
        if preserve_history:
            # Append to existing circuit
            if len(keys) == 1:
                state_obj = self.states[keys[0]]
                state_obj.circuit.append("R", keys)
                for instruction in circuit:
                    state_obj.circuit.append(instruction)
            else:
                # Group if needed
                states = [self.states[k] for k in keys if k in self.states]
                if states and not all(s == states[0] for s in states):
                    self.group_qubits(keys)
                
                state_obj = self.states[keys[0]]
                for key in keys:
                    state_obj.circuit.append("R", [key])
                for instruction in circuit:
                    state_obj.circuit.append(instruction)
        else:
            # Create fresh state
            new_state = StabilizerState(
                keys=keys,
                circuit=circuit.copy() if isinstance(circuit, stim.Circuit) else circuit,
                shots=self.shots,
                truncation=self.truncation
            )
            
            for key in keys:
                self.states[key] = new_state
    
    def _convert_to_stim_circuit(self, state=None, num_qubits: Optional[int] = None) -> stim.Circuit:
        """
        Convert various input types to a Stim circuit.
        
        Args:
            state: Input state (various formats)
            num_qubits: Number of qubits (for amplitude validation)
        
        Returns:
            stim.Circuit that prepares the state
        
        Raises:
            ValueError: If conversion fails
        """
        if state is None:
            # Default to |0...0⟩
            return stim.Circuit()
        
        if isinstance(state, stim.Circuit):
            # Already a Stim circuit
            return state
        
        if isinstance(state, (list, tuple)):
            # Could be amplitude array or density matrix
            if state and isinstance(state[0], (list, tuple)):
                # Density matrix - convert to amplitudes if pure
                state_array = np.array(state, dtype=complex)
                
                # Check if pure state (rank 1)
                eigenvalues = np.linalg.eigvalsh(state_array)
                if np.sum(eigenvalues > 1e-10) == 1:
                    # Pure state - extract amplitudes
                    _, eigenvectors = np.linalg.eigh(state_array)
                    idx = np.argmax(eigenvalues)
                    amplitudes = eigenvectors[:, idx].tolist()
                else:
                    raise ValueError("Mixed states cannot be converted to stabilizer circuits")
            else:
                # Amplitude array
                amplitudes = [complex(a) for a in state]
            
            # Use transpiler
            try:
                return amplitudes_to_stim_circuit(amplitudes, num_qubits)
            except ValueError as e:
                raise ValueError(f"Cannot create stabilizer state: {e}")
        
        raise TypeError(f"Unsupported state type: {type(state)}")
    
    def set_to_zero(self, key: int) -> None:
        """Set qubit to |0⟩ state (SeQUeNCe compatibility)."""
        self.set([key], None)
    
    def set_to_one(self, key: int) -> None:
        """Set qubit to |1⟩ state (SeQUeNCe compatibility)."""
        circuit = stim.Circuit()
        circuit.append("X", [0])
        self.set([key], circuit)
    
    def group_qubits(self, keys: List[int]) -> None:
        """
        Manually group qubits together for joint density matrix computation.
        
        Args:
            keys: List of qubit keys to group together
        """
        if len(keys) <= 1:
            return
        
        # Check if already grouped
        states = [self.states[key] for key in keys if key in self.states]
        if states and all(s == states[0] for s in states):
            return
        
        # Combine circuits from all involved states
        combined_circuit = stim.Circuit()
        all_keys = []
        
        # Collect all unique states and their circuits
        seen_states = set()
        for key in keys:
            if key in self.states:
                state = self.states[key]
                if state not in seen_states:
                    seen_states.add(state)
                    all_keys.extend(state.keys)
                    
                    # Copy operations from this state's circuit
                    for instruction in state.circuit:
                        combined_circuit.append(instruction)
        
        # Create new grouped state
        grouped_state = StabilizerState(
            keys=all_keys,
            circuit=combined_circuit,
            shots=self.shots,
            truncation=self.truncation
        )
        
        # Update all qubits to point to this grouped state
        for key in all_keys:
            self.states[key] = grouped_state
    
    def compute_density_matrix(self, keys: List[int]) -> np.ndarray:
        """
        Compute the density matrix for specific qubits.
        
        Args:
            keys: List of qubit keys to compute density matrix for
        
        Returns:
            Density matrix as numpy array (2^n × 2^n for n qubits)
        """
        if not keys:
            raise ValueError("Must provide at least one key")
        
        # Check if all keys exist
        for key in keys:
            if key not in self.states:
                raise KeyError(f"Key {key} not found in states")
        
        # Check if all requested qubits are already in the same state
        states = [self.states[k] for k in keys]
        if all(s == states[0] for s in states):
            state = states[0]
            if set(state.keys) == set(keys):
                return state.state
        
        # Create a temporary combined circuit for these specific qubits
        combined_circuit = stim.Circuit()
        
        # Collect operations from all involved states
        seen_states = set()
        for key in keys:
            state = self.states[key]
            if state not in seen_states:
                seen_states.add(state)
                for instruction in state.circuit:
                    combined_circuit.append(instruction)
        
        # Create temporary state to compute density matrix
        temp_state = StabilizerState(
            keys=keys,
            circuit=combined_circuit,
            shots=self.shots,
            truncation=self.truncation
        )
        
        return temp_state.state
    
    def get_density_matrix(self, key: int) -> np.ndarray:
        """
        Get the density matrix for a qubit.
        
        Args:
            key: Qubit key
        
        Returns:
            Density matrix as numpy array
        """
        if key not in self.states:
            raise KeyError(f"Key {key} not found in states")
        
        return self.states[key].state
    
    def get_circuit(self, key: int) -> stim.Circuit:
        """
        Get the Stim circuit for a qubit's state.
        
        Args:
            key: Qubit key
        
        Returns:
            Stim circuit for the qubit's state
        """
        if key not in self.states:
            raise KeyError(f"Key {key} not found")
        
        return self.states[key].circuit

In [5]:
"""
Comprehensive test suite for QuantumManagerStabilizer with transpiler functionality.
Tests all methods including amplitude/SeQUeNCe circuit support.
"""

import numpy as np
import stim
from typing import List, Any

# Import the real Circuit class
from sequence.components.circuit import Circuit


def test_initialization_and_configuration():
    """Test manager initialization with different parameters"""
    print("\n" + "="*70)
    print("TEST: Initialization and Configuration")
    print("="*70)
    
    # Test 1: Default initialization
    print("\n1. Default initialization:")
    qm1 = QuantumManagerStabilizer()
    print(f"   Shots: {qm1.shots}")
    print(f"   Truncation: {qm1.truncation}")
    print(f"   Number of states: {len(qm1.states)}")
    print(f"   Next available key: {qm1._least_available}")
    
    assert qm1.shots == 1000  # Updated default
    assert qm1.truncation == 1
    assert len(qm1.states) == 0
    assert qm1._least_available == 0
    print("   ✓ Default parameters verified")
    
    # Test 2: Custom initialization
    print("\n2. Custom initialization with shots=100:")
    qm2 = QuantumManagerStabilizer(shots=100)
    print(f"   Shots: {qm2.shots}")
    assert qm2.shots == 100
    print("   ✓ Custom shots parameter set correctly")
    
    # Test 3: Truncation parameter (dormant but should be stored)
    print("\n3. Truncation parameter:")
    qm3 = QuantumManagerStabilizer(truncation=2)
    print(f"   Truncation: {qm3.truncation}")
    assert qm3.truncation == 2
    print("   ✓ Truncation parameter stored (dormant)")


def test_amplitude_initialization():
    """Test creating qubits from amplitude arrays"""
    print("\n" + "="*70)
    print("TEST: Amplitude Array Initialization")
    print("="*70)
    
    qm = QuantumManagerStabilizer(shots=1000)
    
    # Test 1: |0⟩ state from amplitudes
    print("\n1. Creating |0⟩ from amplitudes:")
    q0 = qm.new([1+0j, 0+0j])
    print(f"   Created qubit {q0} from [1, 0]")
    circuit = qm.get_circuit(q0)
    print(f"   Circuit: {circuit}")
    assert len(circuit) == 0
    print("   ✓ |0⟩ state created correctly")
    
    # Test 2: |1⟩ state from amplitudes
    print("\n2. Creating |1⟩ from amplitudes:")
    q1 = qm.new([0, 1])
    print(f"   Created qubit {q1} from [0, 1]")
    circuit = qm.get_circuit(q1)
    print(f"   Circuit: {circuit}")
    assert "X" in str(circuit)
    print("   ✓ |1⟩ state created correctly")
    
    # Test 3: |+⟩ state from amplitudes
    print("\n3. Creating |+⟩ from amplitudes:")
    q_plus = qm.new([1/np.sqrt(2), 1/np.sqrt(2)])
    print(f"   Created qubit from [1/√2, 1/√2]")
    circuit = qm.get_circuit(q_plus)
    print(f"   Circuit: {circuit}")
    assert "H" in str(circuit)
    print("   ✓ |+⟩ state created correctly")
    
    # Test 4: |-⟩ state from amplitudes
    print("\n4. Creating |-⟩ from amplitudes:")
    q_minus = qm.new([1/np.sqrt(2), -1/np.sqrt(2)])
    circuit = qm.get_circuit(q_minus)
    print(f"   Circuit: {circuit}")
    assert "H" in str(circuit) and "Z" in str(circuit)
    print("   ✓ |-⟩ state created correctly")
    
    # Test 5: SeQUeNCe-style tuple input
    print("\n5. Creating from SeQUeNCe-style tuple:")
    q_tuple = qm.new((complex(1), complex(0)))
    print(f"   Created qubit from (complex(1), complex(0))")
    circuit = qm.get_circuit(q_tuple)
    assert len(circuit) == 0
    print("   ✓ SeQUeNCe tuple format supported")
    
    # Test 6: Non-stabilizer state (should fail)
    print("\n6. Testing non-stabilizer state rejection:")
    try:
        q_bad = qm.new([np.sqrt(0.3), np.sqrt(0.7)])
        assert False, "Should have raised ValueError"
    except ValueError as e:
        print(f"   ✓ Correctly rejected: {e}")


def test_bell_state_initialization():
    """Test creating Bell states from amplitudes"""
    print("\n" + "="*70)
    print("TEST: Bell State Initialization from Amplitudes")
    print("="*70)
    
    qm = QuantumManagerStabilizer(shots=1000)
    
    # Test 1: |Φ+⟩ Bell state
    print("\n1. Creating |Φ+⟩ = (|00⟩ + |11⟩)/√2:")
    bell_phi_plus = [1/np.sqrt(2), 0, 0, 1/np.sqrt(2)]
    
    # First create the circuit to verify it works
    from transpiler import amplitudes_to_stim_circuit
    circuit = amplitudes_to_stim_circuit(bell_phi_plus, 2)
    print(f"   Transpiled circuit: {circuit}")
    
    # Now create via set (which groups two qubits)
    q0 = qm.new()
    q1 = qm.new()
    qm.set([q0, q1], bell_phi_plus)
    
    print(f"   Qubits grouped? {qm.states[q0] == qm.states[q1]}")
    print(f"   State keys: {qm.states[q0].keys}")
    
    assert qm.states[q0] == qm.states[q1]
    assert "H" in str(qm.get_circuit(q0))
    assert "CX" in str(qm.get_circuit(q0))
    print("   ✓ |Φ+⟩ Bell state created")
    
    # Test 2: |Ψ+⟩ Bell state
    print("\n2. Creating |Ψ+⟩ = (|01⟩ + |10⟩)/√2:")
    bell_psi_plus = [0, 1/np.sqrt(2), 1/np.sqrt(2), 0]
    
    q2 = qm.new()
    q3 = qm.new()
    qm.set([q2, q3], bell_psi_plus)
    
    circuit = qm.get_circuit(q2)
    print(f"   Circuit: {circuit}")
    assert "H" in str(circuit) and "CX" in str(circuit) and "X" in str(circuit)
    print("   ✓ |Ψ+⟩ Bell state created")
    
    # Test 3: Product state |++⟩
    print("\n3. Creating product state |++⟩:")
    product_state = [0.5, 0.5, 0.5, 0.5]
    
    q4 = qm.new()
    q5 = qm.new()
    qm.set([q4, q5], product_state)
    
    circuit = qm.get_circuit(q4)
    print(f"   Circuit: {circuit}")
    assert str(circuit).count("H") == 2
    print("   ✓ |++⟩ product state created")


def test_sequence_circuit_transpilation():
    """Test running SeQUeNCe Circuit objects"""
    print("\n" + "="*70)
    print("TEST: SeQUeNCe Circuit Transpilation")
    print("="*70)
    
    qm = QuantumManagerStabilizer(shots=1000)
    
    # Test 1: Basic SeQUeNCe circuit using the real Circuit class
    print("\n1. Running basic SeQUeNCe circuit:")
    q0 = qm.new()
    q1 = qm.new()
    
    # Create a real Circuit object
    seq_circuit = Circuit(2)
    seq_circuit.h(0)
    seq_circuit.cx(0, 1)
    seq_circuit.measure(0)
    seq_circuit.measure(1)
    
    print(f"   SeQUeNCe gates: {seq_circuit.gates}")
    print(f"   Measured qubits: {seq_circuit.measured_qubits}")
    results = qm.run_circuit(seq_circuit, [q0, q1])
    
    print(f"   Measurement results: {results}")
    assert q0 in results and q1 in results
    print("   ✓ SeQUeNCe circuit executed successfully")
    
    # Test 2: SeQUeNCe circuit with S gates
    print("\n2. SeQUeNCe circuit with phase gates:")
    q2 = qm.new()
    
    seq_circuit2 = Circuit(1)
    seq_circuit2.h(0)
    seq_circuit2.s(0)
    seq_circuit2.sdg(0)
    seq_circuit2.z(0)
    
    qm.run_circuit(seq_circuit2, [q2])
    circuit = qm.get_circuit(q2)
    
    print(f"   SeQUeNCe gates: {seq_circuit2.gates}")
    print(f"   Final Stim circuit: {circuit}")
    assert "H" in str(circuit)
    assert "S" in str(circuit)
    assert "S_DAG" in str(circuit)
    assert "Z" in str(circuit)
    print("   ✓ Phase gates transpiled correctly")
    
    # Test 3: Non-Clifford gate rejection (T gate)
    print("\n3. Testing non-Clifford gate rejection:")
    seq_circuit3 = Circuit(1)
    seq_circuit3.h(0)
    seq_circuit3.t(0)  # T gate is non-Clifford
    
    try:
        qm.run_circuit(seq_circuit3, [q2])
        assert False, "Should have raised ValueError"
    except ValueError as e:
        print(f"   ✓ Correctly rejected T gate: {e}")
    
    # Test 4: SWAP gate
    print("\n4. Testing SWAP gate:")
    q3 = qm.new()
    q4 = qm.new()
    
    seq_circuit4 = Circuit(2)
    seq_circuit4.x(0)  # Put q3 in |1⟩
    seq_circuit4.swap(0, 1)  # Swap them
    
    qm.run_circuit(seq_circuit4, [q3, q4])
    circuit = qm.get_circuit(q3)
    
    print(f"   Circuit: {circuit}")
    assert "X" in str(circuit) and "SWAP" in str(circuit)
    print("   ✓ SWAP gate transpiled correctly")
    
    # Test 5: CZ gate
    print("\n5. Testing CZ gate:")
    q5 = qm.new()
    q6 = qm.new()
    
    seq_circuit5 = Circuit(2)
    seq_circuit5.h(0)
    seq_circuit5.h(1)
    seq_circuit5.cz(0, 1)
    
    qm.run_circuit(seq_circuit5, [q5, q6])
    circuit = qm.get_circuit(q5)
    
    print(f"   Circuit: {circuit}")
    assert "CZ" in str(circuit)
    print("   ✓ CZ gate transpiled correctly")


def test_circuit_class_builder_methods():
    """Test using Circuit class builder methods"""
    print("\n" + "="*70)
    print("TEST: Circuit Class Builder Methods")
    print("="*70)
    
    qm = QuantumManagerStabilizer(shots=1000)
    
    # Test 1: Build circuit using method calls
    print("\n1. Building circuit with method calls:")
    q0 = qm.new()
    q1 = qm.new()
    
    circuit = Circuit(2)
    circuit.h(0)
    circuit.x(1)
    circuit.cx(0, 1)
    circuit.s(0)
    circuit.sdg(1)
    circuit.y(0)
    circuit.z(1)
    
    print(f"   Built circuit gates: {circuit.gates}")
    
    qm.run_circuit(circuit, [q0, q1])
    stim_circuit = qm.get_circuit(q0)
    
    print(f"   Resulting Stim circuit: {stim_circuit}")
    assert len(stim_circuit) == 7  # All 7 gates
    print("   ✓ All builder methods work")
    
    # Test 2: Test swap and cz
    print("\n2. Testing swap and cz methods:")
    q2 = qm.new()
    q3 = qm.new()
    
    circuit2 = Circuit(2)
    circuit2.h(0)
    circuit2.h(1)
    circuit2.cz(0, 1)
    circuit2.swap(0, 1)
    
    qm.run_circuit(circuit2, [q2, q3])
    result = qm.get_circuit(q2)
    
    print(f"   Circuit: {result}")
    assert "CZ" in str(result) and "SWAP" in str(result)
    print("   ✓ CZ and SWAP methods work")
    
    # Test 3: Measure method
    print("\n3. Testing measure method:")
    q4 = qm.new()
    
    circuit3 = Circuit(1)
    circuit3.h(0)
    circuit3.measure(0)
    
    print(f"   Gates: {circuit3.gates}")
    print(f"   Measured qubits: {circuit3.measured_qubits}")
    
    results = qm.run_circuit(circuit3, [q4])
    print(f"   Measurement result: {results}")
    
    assert q4 in results
    assert circuit3.measured_qubits == [0]
    print("   ✓ Measure method works")
    
    # Test 4: Complex circuit with measurements
    print("\n4. Complex circuit with measurements:")
    q5 = qm.new()
    q6 = qm.new()
    q7 = qm.new()
    
    bell_circuit = Circuit(3)
    bell_circuit.h(0)
    bell_circuit.cx(0, 1)
    bell_circuit.cx(1, 2)
    bell_circuit.measure(0)
    bell_circuit.measure(1)
    bell_circuit.measure(2)
    
    results = qm.run_circuit(bell_circuit, [q5, q6, q7])
    print(f"   Measurement results: {results}")
    
    # Check GHZ-like correlations
    vals = [results[q5], results[q6], results[q7]]
    print(f"   Values: {vals}")
    assert len(results) == 3
    print("   ✓ Complex circuit with measurements works")


def test_set_with_amplitudes():
    """Test set function with amplitude arrays"""
    print("\n" + "="*70)
    print("TEST: Set Function with Amplitudes")
    print("="*70)
    
    qm = QuantumManagerStabilizer(shots=1000)
    
    # Test 1: Set single qubit to |+⟩
    print("\n1. Set single qubit to |+⟩:")
    q0 = qm.new()
    qm.set([q0], [1/np.sqrt(2), 1/np.sqrt(2)])
    
    circuit = qm.get_circuit(q0)
    print(f"   Circuit after set: {circuit}")
    assert "H" in str(circuit)
    print("   ✓ Set to |+⟩ via amplitudes")
    
    # Test 2: Set to computational basis state |101⟩
    print("\n2. Set three qubits to |101⟩:")
    q1 = qm.new()
    q2 = qm.new()
    q3 = qm.new()
    
    # |101⟩ has amplitude 1 at index 5 (binary 101)
    amplitudes_101 = [0] * 8
    amplitudes_101[5] = 1
    
    qm.set([q1, q2, q3], amplitudes_101)
    circuit = qm.get_circuit(q1)
    
    print(f"   Circuit: {circuit}")
    # Should have X on qubits 0 and 2 (for |101⟩)
    assert str(circuit).count("X") == 2
    print("   ✓ Set to |101⟩ via amplitudes")
    
    # Test 3: Set from density matrix (pure state)
    print("\n3. Set from pure density matrix:")
    q4 = qm.new()
    
    # |+⟩⟨+| density matrix
    rho_plus = [[0.5, 0.5], [0.5, 0.5]]
    qm.set([q4], rho_plus)
    
    circuit = qm.get_circuit(q4)
    print(f"   Circuit from density matrix: {circuit}")
    assert "H" in str(circuit)
    print("   ✓ Set from pure density matrix")
    
    # Test 4: Reject mixed state
    print("\n4. Reject mixed density matrix:")
    mixed_rho = [[0.5, 0], [0, 0.5]]  # Maximally mixed state
    
    try:
        qm.set([q4], mixed_rho)
        assert False, "Should have raised ValueError"
    except ValueError as e:
        print(f"   ✓ Correctly rejected mixed state: {e}")


def test_ghz_state():
    """Test creating and manipulating GHZ states"""
    print("\n" + "="*70)
    print("TEST: GHZ State from Amplitudes")
    print("="*70)
    
    qm = QuantumManagerStabilizer(shots=1000)
    
    # Test 1: Create 3-qubit GHZ from amplitudes
    print("\n1. Creating GHZ state (|000⟩ + |111⟩)/√2:")
    
    ghz_amplitudes = [0] * 8
    ghz_amplitudes[0] = 1/np.sqrt(2)  # |000⟩
    ghz_amplitudes[7] = 1/np.sqrt(2)  # |111⟩
    
    q0 = qm.new()
    q1 = qm.new()
    q2 = qm.new()
    
    qm.set([q0, q1, q2], ghz_amplitudes)
    
    circuit = qm.get_circuit(q0)
    print(f"   GHZ circuit: {circuit}")
    
    assert "H" in str(circuit)
    assert str(circuit).count("CX") >= 2
    assert qm.states[q0] == qm.states[q1] == qm.states[q2]
    print("   ✓ GHZ state created from amplitudes")
    
    # Test 2: Verify it's actually GHZ by measurement statistics
    print("\n2. Verifying GHZ correlations:")
    
    # In GHZ state, all qubits should measure the same
    measurements = []
    for _ in range(10):
        # Reset and recreate GHZ
        qm.set([q0, q1, q2], ghz_amplitudes)
        
        meas_circuit = stim.Circuit()
        meas_circuit.append("M", [0, 1, 2])
        results = qm.run_circuit(meas_circuit, [q0, q1, q2])
        
        # Check if all measurements are same
        vals = list(results.values())
        if vals[0] == vals[1] == vals[2]:
            measurements.append("correlated")
        else:
            measurements.append("uncorrelated")
    
    print(f"   Measurements: {measurements.count('correlated')}/10 correlated")
    assert measurements.count('correlated') == 10
    print("   ✓ GHZ correlations verified")


def test_preserve_history():
    """Test preserve_history option in set function"""
    print("\n" + "="*70)
    print("TEST: Preserve History in Set Function")
    print("="*70)
    
    qm = QuantumManagerStabilizer(shots=1000)
    
    # Test 1: Set with history preservation
    print("\n1. Set with preserve_history=True:")
    q0 = qm.new()
    
    # First apply some gates
    qm.run_circuit(stim.Circuit("H 0\nS 0"), [q0])
    initial_len = len(qm.get_circuit(q0))
    print(f"   Initial circuit length: {initial_len}")
    
    # Set to |1⟩ with history preservation
    qm.set([q0], [0, 1], preserve_history=True)
    
    circuit = qm.get_circuit(q0)
    print(f"   Circuit after set: {circuit}")
    print(f"   Circuit length: {len(circuit)}")
    
    assert "R" in str(circuit)  # Reset gate
    assert "X" in str(circuit)  # X for |1⟩
    assert len(circuit) > initial_len
    print("   ✓ History preserved with reset")
    
    # Test 2: Set without history preservation
    print("\n2. Set without preserve_history (default):")
    q1 = qm.new()
    
    qm.run_circuit(stim.Circuit("H 0\nS 0\nX 0"), [q1])
    print(f"   Initial circuit: {qm.get_circuit(q1)}")
    
    qm.set([q1], [1/np.sqrt(2), 1/np.sqrt(2)])
    
    circuit = qm.get_circuit(q1)
    print(f"   Circuit after set: {circuit}")
    
    assert len(circuit) == 1  # Only H gate
    assert "H" in str(circuit)
    print("   ✓ Circuit replaced (history not preserved)")


def test_compatibility_methods():
    """Test SeQUeNCe compatibility methods"""
    print("\n" + "="*70)
    print("TEST: SeQUeNCe Compatibility Methods")
    print("="*70)
    
    qm = QuantumManagerStabilizer(shots=1000)
    
    # Test 1: set_to_zero
    print("\n1. Testing set_to_zero:")
    q0 = qm.new([0, 1])  # Start in |1⟩
    print(f"   Initial state: |1⟩")
    
    qm.set_to_zero(q0)
    circuit = qm.get_circuit(q0)
    print(f"   After set_to_zero: {circuit}")
    
    assert len(circuit) == 0  # |0⟩ is empty circuit
    print("   ✓ set_to_zero works")
    
    # Test 2: set_to_one
    print("\n2. Testing set_to_one:")
    q1 = qm.new()  # Start in |0⟩
    print(f"   Initial state: |0⟩")
    
    qm.set_to_one(q1)
    circuit = qm.get_circuit(q1)
    print(f"   After set_to_one: {circuit}")
    
    assert "X" in str(circuit)
    print("   ✓ set_to_one works")


def test_mixed_operations():
    """Test mixing amplitude initialization with circuit operations"""
    print("\n" + "="*70)
    print("TEST: Mixed Operations (Amplitudes + Circuits)")
    print("="*70)
    
    qm = QuantumManagerStabilizer(shots=1000)
    
    # Test 1: Initialize with amplitudes, then apply gates
    print("\n1. Initialize |+⟩, then apply gates:")
    q0 = qm.new([1/np.sqrt(2), 1/np.sqrt(2)])
    print(f"   Created |+⟩ from amplitudes")
    
    # Apply additional gates
    circuit = stim.Circuit()
    circuit.append("S", [0])
    circuit.append("Z", [0])
    qm.run_circuit(circuit, [q0])
    
    final_circuit = qm.get_circuit(q0)
    print(f"   Final circuit: {final_circuit}")
    
    assert "H" in str(final_circuit)  # From |+⟩
    assert "S" in str(final_circuit)  # Applied after
    assert "Z" in str(final_circuit)  # Applied after
    print("   ✓ Gates applied after amplitude initialization")
    
    # Test 2: Create Bell state from amplitudes, then measure
    print("\n2. Bell state from amplitudes, then measure:")
    q1 = qm.new()
    q2 = qm.new()
    
    bell = [1/np.sqrt(2), 0, 0, 1/np.sqrt(2)]
    qm.set([q1, q2], bell)
    print(f"   Created Bell state from amplitudes")
    
    # Measure multiple times to check correlation
    correlations = 0
    num_tests = 10
    for _ in range(num_tests):
        # Reset to Bell state
        qm.set([q1, q2], bell)
        
        # Measure
        meas_circuit = stim.Circuit()
        meas_circuit.append("M", [0, 1])
        results = qm.run_circuit(meas_circuit, [q1, q2])
        
        if results[q1] == results[q2]:
            correlations += 1
    
    print(f"   Correlated measurements: {correlations}/{num_tests}")
    assert q1 in results and q2 in results
    # Bell state should have perfect correlation
    assert correlations == num_tests
    print("   ✓ Bell state measured correctly")
    
    # Test 3: Mix SeQUeNCe and Stim circuits
    print("\n3. Mix SeQUeNCe and Stim circuits:")
    q3 = qm.new()
    
    # First use SeQUeNCe circuit
    seq_circuit = Circuit(1)
    seq_circuit.h(0)
    qm.run_circuit(seq_circuit, [q3])
    
    # Then use Stim circuit
    stim_circuit = stim.Circuit()
    stim_circuit.append("S", [0])
    qm.run_circuit(stim_circuit, [q3])
    
    final = qm.get_circuit(q3)
    print(f"   Final circuit: {final}")
    
    assert "H" in str(final) and "S" in str(final)
    print("   ✓ SeQUeNCe and Stim circuits mixed successfully")
    
    # Test 4: Create state with Circuit class, then apply Stim
    print("\n4. Circuit class followed by Stim operations:")
    q4 = qm.new()
    q5 = qm.new()
    
    # Use Circuit class to create Bell state
    seq_bell = Circuit(2)
    seq_bell.h(0)
    seq_bell.cx(0, 1)
    qm.run_circuit(seq_bell, [q4, q5])
    
    # Apply Stim circuit on top
    stim_ops = stim.Circuit()
    stim_ops.append("Z", [0])
    stim_ops.append("Z", [1])
    qm.run_circuit(stim_ops, [q4, q5])
    
    final = qm.get_circuit(q4)
    print(f"   Final circuit: {final}")
    
    assert "H" in str(final) and "CX" in str(final) and "Z" in str(final)
    print("   ✓ Circuit class and Stim operations combined")


def test_error_messages():
    """Test that error messages are clear and helpful"""
    print("\n" + "="*70)
    print("TEST: Error Message Clarity")
    print("="*70)
    
    qm = QuantumManagerStabilizer(shots=1000)
    
    # Test 1: Non-normalized amplitudes
    print("\n1. Non-normalized amplitude error:")
    try:
        qm.new([1, 1])  # Not normalized
    except ValueError as e:
        print(f"   Error: {e}")
        assert "not normalized" in str(e).lower()
        print("   ✓ Clear normalization error")
    
    # Test 2: Wrong length amplitudes
    print("\n2. Wrong length amplitude error:")
    try:
        qm.new([1, 0, 0])  # Length 3, not power of 2
    except ValueError as e:
        print(f"   Error: {e}")
        assert "power of 2" in str(e).lower()
        print("   ✓ Clear length error")
    
    # Test 3: Non-stabilizer state
    print("\n3. Non-stabilizer state error:")
    try:
        qm.new([np.cos(np.pi/8), np.sin(np.pi/8)])  # Not stabilizer
    except ValueError as e:
        print(f"   Error: {e}")
        assert "not a stabilizer state" in str(e).lower()
        print("   ✓ Clear non-stabilizer error")
    
    # Test 4: Non-Clifford gate in SeQUeNCe Circuit
    print("\n4. Non-Clifford gate error (phase gate):")
    seq = Circuit(1)
    seq.phase(0, np.pi/8)  # Phase gate with non-Clifford angle
    
    try:
        qm.run_circuit(seq, [qm.new()])
    except ValueError as e:
        print(f"   Error: {e}")
        assert "not clifford" in str(e).lower() or "phase" in str(e).lower()
        print("   ✓ Clear non-Clifford error")
    
    # Test 5: Toffoli gate (CCX) rejection
    print("\n5. Toffoli gate (non-Clifford) error:")
    seq2 = Circuit(3)
    seq2.ccx(0, 1, 2)  # Toffoli is not Clifford
    
    try:
        qm.run_circuit(seq2, [qm.new(), qm.new(), qm.new()])
    except ValueError as e:
        print(f"   Error: {e}")
        assert "ccx" in str(e).lower() or "not clifford" in str(e).lower()
        print("   ✓ Clear Toffoli rejection error")


def run_comprehensive_tests():
    """Run all comprehensive tests for enhanced QuantumManagerStabilizer"""
    print("\n" + "="*70)
    print("COMPREHENSIVE TEST SUITE FOR QuantumManagerStabilizer")
    print("WITH TRANSPILER FUNCTIONALITY")
    print("="*70)
    
    test_functions = [
        test_initialization_and_configuration,
        test_amplitude_initialization,
        test_bell_state_initialization,
        test_sequence_circuit_transpilation,
        test_circuit_class_builder_methods,
        test_set_with_amplitudes,
        test_ghz_state,
        test_preserve_history,
        test_compatibility_methods,
        test_mixed_operations,
        test_error_messages,
    ]
    
    passed = 0
    failed = 0
    
    for test_func in test_functions:
        try:
            test_func()
            passed += 1
            print(f"\n✓✓✓ {test_func.__name__} PASSED ✓✓✓")
        except Exception as e:
            failed += 1
            print(f"\n✗✗✗ {test_func.__name__} FAILED: {e} ✗✗✗")
            import traceback
            traceback.print_exc()
    
    print("\n" + "="*70)
    print("FINAL TEST SUMMARY")
    print("="*70)
    print(f"Tests Passed: {passed}/{len(test_functions)}")
    print(f"Tests Failed: {failed}/{len(test_functions)}")
    
    if failed == 0:
        print("\n🎉 ALL TESTS PASSED SUCCESSFULLY! 🎉")
        print("QuantumManagerStabilizer with transpiler support is fully functional!")
    else:
        print(f"\n⚠️  {failed} TEST(S) FAILED - REVIEW OUTPUT ABOVE ⚠️")
    
    return passed, failed


if __name__ == "__main__":
    passed, failed = run_comprehensive_tests()
    
    # Exit with appropriate code
    import sys
    sys.exit(0 if failed == 0 else 1)


COMPREHENSIVE TEST SUITE FOR QuantumManagerStabilizer
WITH TRANSPILER FUNCTIONALITY

TEST: Initialization and Configuration

1. Default initialization:
   Shots: 1000
   Truncation: 1
   Number of states: 0
   Next available key: 0
   ✓ Default parameters verified

2. Custom initialization with shots=100:
   Shots: 100
   ✓ Custom shots parameter set correctly

3. Truncation parameter:
   Truncation: 2
   ✓ Truncation parameter stored (dormant)

✓✓✓ test_initialization_and_configuration PASSED ✓✓✓

TEST: Amplitude Array Initialization

1. Creating |0⟩ from amplitudes:
   Created qubit 0 from [1, 0]
   Circuit: 
   ✓ |0⟩ state created correctly

2. Creating |1⟩ from amplitudes:
   Created qubit 1 from [0, 1]
   Circuit: X 0
   ✓ |1⟩ state created correctly

3. Creating |+⟩ from amplitudes:
   Created qubit from [1/√2, 1/√2]
   Circuit: H 0
   ✓ |+⟩ state created correctly

4. Creating |-⟩ from amplitudes:
   Circuit: H 0
Z 0
   ✓ |-⟩ state created correctly

5. Creating from SeQUeNCe-s

Traceback (most recent call last):
  File "C:\Users\PatEliteBook\AppData\Local\Temp\ipykernel_13732\309697012.py", line 721, in run_comprehensive_tests
    test_func()
  File "C:\Users\PatEliteBook\AppData\Local\Temp\ipykernel_13732\309697012.py", line 122, in test_bell_state_initialization
    from stim_sequence_transpilers import amplitudes_to_stim_circuit
ModuleNotFoundError: No module named 'stim_sequence_transpilers'


   SeQUeNCe gates: [['h', [0], None], ['s', [0], None], ['sdg', [0], None], ['z', [0], None]]
   Final Stim circuit: H 2
S 2
S_DAG 2
Z 2
   ✓ Phase gates transpiled correctly

3. Testing non-Clifford gate rejection:
   ✓ Correctly rejected T gate: Cannot run SeQUeNCe circuit on stabilizer manager: Gate 't' is not Clifford and cannot be used in Stim

4. Testing SWAP gate:
   Circuit: X 3
SWAP 3 4
   ✓ SWAP gate transpiled correctly

5. Testing CZ gate:
   Circuit: H 5 6
CZ 5 6
   ✓ CZ gate transpiled correctly

✓✓✓ test_sequence_circuit_transpilation PASSED ✓✓✓

TEST: Circuit Class Builder Methods

1. Building circuit with method calls:
   Built circuit gates: [['h', [0], None], ['x', [1], None], ['cx', [0, 1], None], ['s', [0], None], ['sdg', [1], None], ['y', [0], None], ['z', [1], None]]
   Resulting Stim circuit: H 0
X 1
CX 0 1
S 0
S_DAG 1
Y 0
Z 1
   ✓ All builder methods work

2. Testing swap and cz methods:
   Circuit: H 2 3
CZ 2 3
SWAP 2 3
   ✓ CZ and SWAP methods work

3. Testin

Traceback (most recent call last):
  File "C:\Users\PatEliteBook\AppData\Local\Temp\ipykernel_13732\309697012.py", line 721, in run_comprehensive_tests
    test_func()
  File "C:\Users\PatEliteBook\AppData\Local\Temp\ipykernel_13732\309697012.py", line 385, in test_set_with_amplitudes
    assert str(circuit).count("X") == 2
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError
Traceback (most recent call last):
  File "C:\Users\PatEliteBook\AppData\Local\Temp\ipykernel_13732\309697012.py", line 721, in run_comprehensive_tests
    test_func()
  File "C:\Users\PatEliteBook\AppData\Local\Temp\ipykernel_13732\309697012.py", line 437, in test_ghz_state
    assert str(circuit).count("CX") >= 2
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError
Traceback (most recent call last):
  File "C:\Users\PatEliteBook\AppData\Local\Temp\ipykernel_13732\309697012.py", line 721, in run_comprehensive_tests
    test_func()
  File "C:\Users\PatEliteBook\AppData\Local\Temp\ipykernel_13732\309697012.py"

   Error: Cannot run SeQUeNCe circuit on stabilizer manager: Gate 'phase' is not Clifford and cannot be used in Stim
   ✓ Clear non-Clifford error

5. Toffoli gate (non-Clifford) error:
   Error: Cannot run SeQUeNCe circuit on stabilizer manager: Gate 'ccx' is not supported
   ✓ Clear Toffoli rejection error

✓✓✓ test_error_messages PASSED ✓✓✓

FINAL TEST SUMMARY
Tests Passed: 7/11
Tests Failed: 4/11

⚠️  4 TEST(S) FAILED - REVIEW OUTPUT ABOVE ⚠️


SystemExit: 1

c:\Users\PatEliteBook\anaconda3\envs\gridq\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [69]:
"""
Comprehensive test suite for the entire QuantumManagerStabilizer class.
Tests all methods with detailed assertions and informative print statements.
"""

import numpy as np
import stim



def test_initialization_and_configuration():
    """Test manager initialization with different parameters"""
    print("\n" + "="*70)
    print("TEST: Initialization and Configuration")
    print("="*70)
    
    # Test 1: Default initialization
    print("\n1. Default initialization:")
    qm1 = QuantumManagerStabilizer()
    print(f"   Shots: {qm1.shots}")
    print(f"   Truncation: {qm1.truncation}")
    print(f"   Number of states: {len(qm1.states)}")
    print(f"   Next available key: {qm1._least_available}")
    
    assert qm1.shots == 8192
    assert qm1.truncation == 1
    assert len(qm1.states) == 0
    assert qm1._least_available == 0
    print("   ✓ Default parameters verified")
    
    # Test 2: Custom initialization
    print("\n2. Custom initialization with shots=100:")
    qm2 = QuantumManagerStabilizer(shots=100)
    print(f"   Shots: {qm2.shots}")
    assert qm2.shots == 100
    print("   ✓ Custom shots parameter set correctly")
    
    # Test 3: Truncation parameter (dormant but should be stored)
    print("\n3. Truncation parameter:")
    qm3 = QuantumManagerStabilizer(truncation=2)
    print(f"   Truncation: {qm3.truncation}")
    assert qm3.truncation == 2
    print("   ✓ Truncation parameter stored (dormant)")


def test_qubit_creation_and_management():
    """Test creating, accessing, and managing qubits"""
    print("\n" + "="*70)
    print("TEST: Qubit Creation and Management")
    print("="*70)
    
    qm = QuantumManagerStabilizer(shots=1000)
    
    # Test 1: Create single qubit
    print("\n1. Creating single qubit:")
    q0 = qm.new()
    print(f"   Created qubit with key: {q0}")
    print(f"   State keys: {qm.states[q0].keys}")
    print(f"   Circuit: {qm.get_circuit(q0)}")
    
    assert q0 == 0
    assert qm.states[q0].keys == [0]
    assert len(qm.get_circuit(q0)) == 0
    print("   ✓ Qubit created in |0⟩ state")
    
    # Test 2: Create with initial circuit
    print("\n2. Creating qubit with initial circuit:")
    init_circuit = stim.Circuit()
    init_circuit.append("X", [0])
    q1 = qm.new(state=init_circuit)
    print(f"   Created qubit with key: {q1}")
    print(f"   Initial circuit: {qm.get_circuit(q1)}")
    
    assert q1 == 1
    assert "X" in str(qm.get_circuit(q1))
    print("   ✓ Qubit created with initial X gate")
    
    # Test 3: Invalid initial state type
    print("\n3. Testing invalid initial state type:")
    try:
        q_bad = qm.new(state="invalid")
        assert False, "Should have raised TypeError"
    except TypeError as e:
        print(f"   ✓ Correctly raised TypeError: {e}")
    
    # Test 4: Multiple qubit creation
    print("\n4. Creating multiple qubits:")
    print(qm)
    qubits = [qm.new() for _ in range(3)]
    print(f"   Created qubits with keys: {qubits}")
    print(f"   Total qubits in manager: {qm._least_available}")
    
    assert qubits == [2, 3, 4]
    assert qm._least_available == 5
    print("   ✓ Multiple qubits created with sequential keys")


def test_single_qubit_gates():
    """Test applying various single-qubit gates"""
    print("\n" + "="*70)
    print("TEST: Single-Qubit Gate Operations")
    print("="*70)
    
    qm = QuantumManagerStabilizer(shots=1000)
    q0 = qm.new()
    
    gates = ["H", "X", "Y", "Z", "S", "S_DAG"]
    
    print("\n1. Applying single-qubit gates sequentially:")
    for gate in gates:
        circuit = stim.Circuit()
        circuit.append(gate, [0])
        qm.run_circuit(circuit, [q0])
        print(f"   Applied {gate}, circuit length: {len(qm.get_circuit(q0))}")
    
    final_circuit = qm.get_circuit(q0)
    print(f"   Final circuit: {final_circuit}")
    assert len(final_circuit) == len(gates)
    print("   ✓ All gates applied successfully")
    
    # Test 2: Apply multiple gates in one circuit
    print("\n2. Applying multiple gates in single circuit:")
    q1 = qm.new()
    circuit = stim.Circuit()
    circuit.append("H", [0])
    circuit.append("S", [0])
    circuit.append("X", [0])
    
    qm.run_circuit(circuit, [q1])
    print(f"   Applied H-S-X sequence")
    print(f"   Resulting circuit: {qm.get_circuit(q1)}")
    
    assert len(qm.get_circuit(q1)) == 3
    print("   ✓ Multiple gates in single circuit applied correctly")


def test_two_qubit_gates_and_grouping():
    """Test two-qubit gates and automatic grouping"""
    print("\n" + "="*70)
    print("TEST: Two-Qubit Gates and Automatic Grouping")
    print("="*70)
    
    qm = QuantumManagerStabilizer(shots=1000)
    
    # Test 1: CNOT gate causes grouping
    print("\n1. CNOT gate automatic grouping:")
    q0 = qm.new()
    q1 = qm.new()
    print(f"   Created separate qubits: q0={q0}, q1={q1}")
    print(f"   Initially grouped? {qm.states[q0] == qm.states[q1]}")
    
    circuit = stim.Circuit()
    circuit.append("CNOT", [0, 1])
    qm.run_circuit(circuit, [q0, q1])
    
    print(f"   After CNOT:")
    print(f"   - Grouped? {qm.states[q0] == qm.states[q1]}")
    print(f"   - State keys: {qm.states[q0].keys}")
    print(f"   - Circuit: {qm.get_circuit(q0)}")
    
    assert qm.states[q0] == qm.states[q1]
    assert set(qm.states[q0].keys) == {0, 1}
    print("   ✓ CNOT automatically grouped qubits")
    
    # Test 2: Other two-qubit gates
    print("\n2. Testing other two-qubit gates:")
    two_qubit_gates = ["CZ", "SWAP"]
    
    for gate in two_qubit_gates:
        q2 = qm.new()
        q3 = qm.new()
        circuit = stim.Circuit()
        circuit.append(gate, [0, 1])
        qm.run_circuit(circuit, [q2, q3])
        print(f"   {gate}: grouped={qm.states[q2] == qm.states[q3]}")
        assert qm.states[q2] == qm.states[q3]
    
    print("   ✓ All two-qubit gates cause grouping")


def test_manual_grouping():
    """Test manual grouping functionality"""
    print("\n" + "="*70)
    print("TEST: Manual Grouping")
    print("="*70)
    
    qm = QuantumManagerStabilizer(shots=1000)
    
    # Test 1: Group two separate qubits
    print("\n1. Manually group two qubits:")
    q0 = qm.new()
    q1 = qm.new()
    
    # Apply different gates to each
    qm.run_circuit(stim.Circuit("H 0"), [q0])
    qm.run_circuit(stim.Circuit("X 0"), [q1])
    
    print(f"   q0 circuit before grouping: {qm.get_circuit(q0)}")
    print(f"   q1 circuit before grouping: {qm.get_circuit(q1)}")
    
    qm.group_qubits([q0, q1])
    
    print(f"   After grouping:")
    print(f"   - Same state? {qm.states[q0] == qm.states[q1]}")
    print(f"   - Combined circuit: {qm.get_circuit(q0)}")
    print(f"   - State keys: {qm.states[q0].keys}")
    
    assert qm.states[q0] == qm.states[q1]
    assert set(qm.states[q0].keys) == {0, 1}
    print("   ✓ Manual grouping successful")
    
    # Test 2: Group already grouped qubits (should be no-op)
    print("\n2. Group already grouped qubits:")
    initial_id = id(qm.states[q0])
    qm.group_qubits([q0, q1])
    final_id = id(qm.states[q0])
    
    print(f"   State object ID before: {initial_id}")
    print(f"   State object ID after: {final_id}")
    print(f"   Same object? {initial_id == final_id}")
    
    assert initial_id == final_id
    print("   ✓ Already grouped qubits not regrouped")
    
    # Test 3: Group three qubits
    print("\n3. Group three qubits:")
    q2 = qm.new()
    qm.group_qubits([q0, q1, q2])
    
    print(f"   All three grouped? {qm.states[q0] == qm.states[q1] == qm.states[q2]}")
    print(f"   State keys: {qm.states[q0].keys}")
    
    assert all(qm.states[q] == qm.states[q0] for q in [q0, q1, q2])
    assert set(qm.states[q0].keys) == {0, 1, 2}
    print("   ✓ Three qubits grouped successfully")


def test_set_function():
    """Test the set function with various options"""
    print("\n" + "="*70)
    print("TEST: Set Function")
    print("="*70)
    
    qm = QuantumManagerStabilizer(shots=1000)
    
    # Test 1: Set to |0⟩ (default)
    print("\n1. Set to |0⟩ state:")
    q0 = qm.new()
    qm.run_circuit(stim.Circuit("H 0"), [q0])
    print(f"   Before set: {qm.get_circuit(q0)}")
    
    qm.set([q0])
    print(f"   After set: {qm.get_circuit(q0)}")
    
    assert len(qm.get_circuit(q0)) == 0
    print("   ✓ Set to |0⟩ successful")
    
    # Test 2: Set to custom state
    print("\n2. Set to |1⟩ state:")
    circuit = stim.Circuit()
    circuit.append("X", [0])
    qm.set([q0], circuit)
    
    print(f"   Circuit after set: {qm.get_circuit(q0)}")
    assert "X" in str(qm.get_circuit(q0))
    print("   ✓ Set to |1⟩ successful")
    
    # Test 3: Preserve history
    print("\n3. Set with preserve_history=True:")
    qm.run_circuit(stim.Circuit("H 0"), [q0])
    print(f"   Before set: {qm.get_circuit(q0)}")
    
    circuit = stim.Circuit("S 0")
    qm.set([q0], circuit, preserve_history=True)
    print(f"   After set with history: {qm.get_circuit(q0)}")
    
    assert "R" in str(qm.get_circuit(q0))
    assert "S" in str(qm.get_circuit(q0))
    print("   ✓ History preserved with reset gate")
    
    # Test 4: Set multiple qubits
    print("\n4. Set multiple qubits to Bell state:")
    q1 = qm.new()
    bell_circuit = stim.Circuit()
    bell_circuit.append("H", [0])
    bell_circuit.append("CNOT", [0, 1])
    
    qm.set([q0, q1], bell_circuit)
    print(f"   Circuit: {qm.get_circuit(q0)}")
    print(f"   Grouped? {qm.states[q0] == qm.states[q1]}")
    
    assert qm.states[q0] == qm.states[q1]
    print("   ✓ Multiple qubits set and grouped")


def test_measurements():
    """Test measurement functionality"""
    print("\n" + "="*70)
    print("TEST: Measurements")
    print("="*70)
    
    qm = QuantumManagerStabilizer(shots=1000)
    
    # Test 1: Single qubit measurement
    print("\n1. Single qubit measurement:")
    q0 = qm.new()
    qm.run_circuit(stim.Circuit("H 0"), [q0])
    print(f"   Applied H gate: {qm.get_circuit(q0)}")
    
    circuit = stim.Circuit()
    circuit.append("M", [0])
    results = qm.run_circuit(circuit, [q0])
    
    print(f"   Measurement result: {results}")
    print(f"   Post-measurement circuit: {qm.get_circuit(q0)}")
    
    assert q0 in results
    assert results[q0] in [0, 1]
    assert len(qm.get_circuit(q0)) <= 1  # Empty or X gate
    print("   ✓ Single qubit measured successfully")
    
    # Test 2: Multi-qubit measurement
    print("\n2. Multi-qubit measurement:")
    q1 = qm.new()
    q2 = qm.new()
    
    # Create GHZ state
    circuit = stim.Circuit()
    circuit.append("H", [0])
    circuit.append("CNOT", [0, 1])
    circuit.append("CNOT", [0, 2])
    qm.run_circuit(circuit, [q1, q2, qm.new()])
    
    print(f"   Created GHZ state")
    print(f"   Grouped? {qm.states[q1] == qm.states[q2]}")
    
    # Measure all
    meas_circuit = stim.Circuit()
    meas_circuit.append("M", [0, 1])
    results = qm.run_circuit(meas_circuit, [q1, q2])
    
    print(f"   Measurement results: {results}")
    print(f"   q1 state keys after: {qm.states[q1].keys}")
    print(f"   q2 state keys after: {qm.states[q2].keys}")
    
    assert q1 in results and q2 in results
    assert qm.states[q1].keys == [q1]  # Separated after measurement
    assert qm.states[q2].keys == [q2]
    print("   ✓ Multi-qubit measurement successful")
    
    # Test 3: Partial measurement
    print("\n3. Partial measurement of entangled state:")
    q3 = qm.new()
    q4 = qm.new()
    
    # Entangle them
    circuit = stim.Circuit()
    circuit.append("H", [0])
    circuit.append("CNOT", [0, 1])
    qm.run_circuit(circuit, [q3, q4])
    
    print(f"   Created Bell pair: {qm.states[q3].keys}")
    
    # Measure only one
    meas_circuit = stim.Circuit()
    meas_circuit.append("M", [0])
    results = qm.run_circuit(meas_circuit, [q3])
    
    print(f"   Measured q3: {results}")
    print(f"   q3 state keys after: {qm.states[q3].keys}")
    
    assert q3 in results
    assert qm.states[q3].keys == [q3]
    print("   ✓ Partial measurement successful")


def test_density_matrix_computation():
    """Test density matrix computation functions"""
    print("\n" + "="*70)
    print("TEST: Density Matrix Computation")
    print("="*70)
    
    qm = QuantumManagerStabilizer(shots=5000)
    
    # Test 1: Single qubit density matrices
    print("\n1. Single qubit density matrices:")
    q0 = qm.new()
    
    # |0⟩ state
    rho_0 = qm.get_density_matrix(q0)
    print(f"   |0⟩ state shape: {rho_0.shape}")
    print(f"   |0⟩ state:\n{rho_0.round(3)}")
    
    expected_0 = np.array([[1, 0], [0, 0]], dtype=complex)
    assert np.allclose(rho_0, expected_0, atol=0.1)
    print("   ✓ |0⟩ density matrix correct")
    
    # |+⟩ state
    qm.run_circuit(stim.Circuit("H 0"), [q0])
    rho_plus = qm.get_density_matrix(q0)
    print(f"   |+⟩ state:\n{rho_plus.real.round(3)}")
    
    expected_plus = np.array([[0.5, 0.5], [0.5, 0.5]], dtype=complex)
    assert np.allclose(rho_plus, expected_plus, atol=0.1)
    print("   ✓ |+⟩ density matrix correct")
    
    # Test 2: Multi-qubit density matrix
    print("\n2. Multi-qubit density matrix:")
    q1 = qm.new()
    
    # Create Bell state
    circuit = stim.Circuit()
    circuit.append("H", [0])
    circuit.append("CNOT", [0, 1])
    qm.run_circuit(circuit, [q0, q1])
    
    rho_bell = qm.get_density_matrix(q0)
    print(f"   Bell state shape: {rho_bell.shape}")
    print(f"   Trace: {np.trace(rho_bell).real:.3f}")
    
    assert rho_bell.shape == (4, 4)
    assert np.abs(np.trace(rho_bell) - 1.0) < 0.01
    print("   ✓ Bell state density matrix correct shape and trace")
    
    # Test 3: compute_density_matrix for specific qubits
    print("\n3. Compute density matrix for specific qubits:")
    q2 = qm.new()
    
    # Compute for ungrouped qubits
    rho_01 = qm.compute_density_matrix([q0, q1])
    print(f"   Density matrix for q0,q1: shape {rho_01.shape}")
    
    rho_02 = qm.compute_density_matrix([q0, q2])
    print(f"   Density matrix for q0,q2: shape {rho_02.shape}")
    
    assert rho_01.shape == (4, 4)
    assert rho_02.shape == (4, 4)
    print("   ✓ Computed density matrices for various qubit combinations")


def test_circuit_operations():
    """Test circuit retrieval and manipulation"""
    print("\n" + "="*70)
    print("TEST: Circuit Operations")
    print("="*70)
    
    qm = QuantumManagerStabilizer(shots=1000)
    
    # Test 1: Empty circuit
    print("\n1. Empty circuit for new qubit:")
    q0 = qm.new()
    circuit = qm.get_circuit(q0)
    print(f"   Circuit: {circuit}")
    print(f"   Length: {len(circuit)}")
    
    assert isinstance(circuit, stim.Circuit)
    assert len(circuit) == 0
    print("   ✓ Empty circuit retrieved correctly")
    
    # Test 2: Circuit building
    print("\n2. Circuit building through operations:")
    operations = [
        ("H", [0]),
        ("S", [0]),
        ("X", [0]),
        ("Z", [0])
    ]
    
    for op, targets in operations:
        c = stim.Circuit()
        c.append(op, targets)
        qm.run_circuit(c, [q0])
        print(f"   Added {op}, circuit length: {len(qm.get_circuit(q0))}")
    
    final = qm.get_circuit(q0)
    print(f"   Final circuit: {final}")
    
    assert len(final) == len(operations)
    print("   ✓ Circuit built correctly")
    
    # Test 3: Circuit for grouped qubits
    print("\n3. Circuit for grouped qubits:")
    q1 = qm.new()
    
    circuit = stim.Circuit()
    circuit.append("CNOT", [0, 1])
    qm.run_circuit(circuit, [q0, q1])
    
    circuit_q0 = qm.get_circuit(q0)
    circuit_q1 = qm.get_circuit(q1)
    
    print(f"   q0 circuit: {circuit_q0}")
    print(f"   q1 circuit: {circuit_q1}")
    print(f"   Same circuit object? {circuit_q0 is circuit_q1}")
    
    assert circuit_q0 is circuit_q1
    print("   ✓ Grouped qubits share same circuit")


def test_complex_scenarios():
    """Test complex realistic quantum computing scenarios"""
    print("\n" + "="*70)
    print("TEST: Complex Scenarios")
    print("="*70)
    
    qm = QuantumManagerStabilizer(shots=5000)
    
    # Scenario 1: Quantum teleportation setup
    print("\n1. Quantum teleportation scenario:")
    
    # Alice's qubit to teleport
    alice_data = qm.new()
    qm.run_circuit(stim.Circuit("H 0"), [alice_data])
    qm.run_circuit(stim.Circuit("S 0"), [alice_data])
    print(f"   Alice's data qubit prepared")
    
    # Create Bell pair between Alice and Bob
    alice_bell = qm.new()
    bob_bell = qm.new()
    bell_circuit = stim.Circuit()
    bell_circuit.append("H", [0])
    bell_circuit.append("CNOT", [0, 1])
    qm.run_circuit(bell_circuit, [alice_bell, bob_bell])
    print(f"   Bell pair created between Alice and Bob")
    
    # Alice's Bell measurement setup
    alice_circuit = stim.Circuit()
    alice_circuit.append("CNOT", [0, 1])
    alice_circuit.append("H", [0])
    qm.run_circuit(alice_circuit, [alice_data, alice_bell])
    
    print(f"   Alice's operations complete")
    print(f"   All three qubits grouped? {qm.states[alice_data] == qm.states[bob_bell]}")
    
    assert qm.states[alice_data] == qm.states[alice_bell] == qm.states[bob_bell]
    print("   ✓ Teleportation circuit prepared correctly")
    
    # Scenario 2: Error correction code (3-qubit repetition)
    print("\n2. Three-qubit repetition code:")
    
    qm = QuantumManagerStabilizer(shots=5000)

    # Logical qubit encoded in 3 physical qubits
    q0 = qm.new()
    q1 = qm.new()
    q2 = qm.new()
    
    # Encode |+⟩ state
    encode_circuit = stim.Circuit()
    encode_circuit.append("H", [0])
    encode_circuit.append("CNOT", [0, 1])
    encode_circuit.append("CNOT", [0, 2])
    qm.run_circuit(encode_circuit, [q0, q1, q2])
    
    print(f"   Encoded |+⟩ in repetition code")
    print(f"   All qubits grouped? {qm.states[q0] == qm.states[q1] == qm.states[q2]}")
    print(f"   Circuit: {qm.get_circuit(q0)}")

    
    # Add error
    error_circuit = stim.Circuit()
    error_circuit.append("X", [0]) 
    qm.run_circuit(error_circuit, [q1])
    
    print(f"   Added X error on middle qubit")
    print(f"   Circuit after error: {qm.get_circuit(q0)}")
    
    print(len(qm.get_circuit(q0)))

    assert len(qm.get_circuit(q0)) == 3  
    print("   ✓ Error correction code simulation successful")
    
    # Scenario 3: Parallel quantum operations
    print("\n3. Parallel operations on separate qubit groups:")
    
    # Group 1: Bell pair
    g1_q0 = qm.new()
    g1_q1 = qm.new()
    qm.run_circuit(stim.Circuit("H 0\nCNOT 0 1"), [g1_q0, g1_q1])
    
    # Group 2: GHZ state
    g2_q0 = qm.new()
    g2_q1 = qm.new()
    g2_q2 = qm.new()
    ghz_circuit = stim.Circuit()
    ghz_circuit.append("H", [0])
    ghz_circuit.append("CNOT", [0, 1])
    ghz_circuit.append("CNOT", [1, 2])
    qm.run_circuit(ghz_circuit, [g2_q0, g2_q1, g2_q2])
    
    print(f"   Group 1 (Bell): keys {qm.states[g1_q0].keys}")
    print(f"   Group 2 (GHZ): keys {qm.states[g2_q0].keys}")
    print(f"   Groups separate? {qm.states[g1_q0] != qm.states[g2_q0]}")
    
    assert qm.states[g1_q0] != qm.states[g2_q0]
    assert len(qm.states[g1_q0].keys) == 2
    assert len(qm.states[g2_q0].keys) == 3
    print("   ✓ Parallel quantum operations maintained separately")


def test_edge_cases_and_errors():
    """Test edge cases and error handling"""
    print("\n" + "="*70)
    print("TEST: Edge Cases and Error Handling")
    print("="*70)
    
    qm = QuantumManagerStabilizer()
    
    # Test 1: Empty circuit
    print("\n1. Running empty circuit:")
    q0 = qm.new()
    empty_circuit = stim.Circuit()
    result = qm.run_circuit(empty_circuit, [q0])
    
    print(f"   Result: {result}")
    print(f"   Circuit unchanged: {len(qm.get_circuit(q0)) == 0}")
    
    assert result == {}
    assert len(qm.get_circuit(q0)) == 0
    print("   ✓ Empty circuit handled correctly")
    
    # Test 2: Invalid key access
    print("\n2. Accessing invalid key:")
    try:
        rho = qm.get_density_matrix(999)
        assert False, "Should have raised KeyError"
    except KeyError as e:
        print(f"   ✓ Correctly raised KeyError: {e}")
    
    # Test 3: Set with invalid type
    print("\n3. Set with invalid type:")
    try:
        qm.set([q0], "not a circuit")
        assert False, "Should have raised TypeError"
    except TypeError as e:
        print(f"   ✓ Correctly raised TypeError: {e}")
    
    # Test 4: compute_density_matrix with empty keys
    print("\n4. Compute density matrix with empty keys:")
    try:
        rho = qm.compute_density_matrix([])
        assert False, "Should have raised ValueError"
    except ValueError as e:
        print(f"   ✓ Correctly raised ValueError: {e}")
    
    # Test 5: Large circuit
    print("\n5. Large circuit with many gates:")
    q1 = qm.new()
    large_circuit = stim.Circuit()
    for _ in range(100):
        large_circuit.append("H", [0])
        large_circuit.append("S", [0])
    
    qm.run_circuit(large_circuit, [q1])
    print(f"   Applied 200 gates")
    print(f"   Circuit length: {len(qm.get_circuit(q1))}")
    
    assert len(qm.get_circuit(q1)) == 200
    print("   ✓ Large circuit handled successfully")


def run_comprehensive_tests():
    """Run all comprehensive tests for QuantumManagerStabilizer"""
    print("\n" + "="*70)
    print("COMPREHENSIVE TEST SUITE FOR QuantumManagerStabilizer")
    print("="*70)
    
    test_functions = [
        # test_initialization_and_configuration,
        # test_qubit_creation_and_management,
        # test_single_qubit_gates,
        # test_two_qubit_gates_and_grouping,
        # test_manual_grouping,
        # test_set_function,
        # test_measurements,
        # test_density_matrix_computation,
        # test_circuit_operations,
        test_complex_scenarios,
        # test_edge_cases_and_errors,
    ]
    
    passed = 0
    failed = 0
    
    for test_func in test_functions:
        try:
            test_func()
            passed += 1
            print(f"\n✓✓✓ {test_func.__name__} PASSED ✓✓✓")
        except Exception as e:
            failed += 1
            print(f"\n✗✗✗ {test_func.__name__} FAILED: {e} ✗✗✗")
            import traceback
            traceback.print_exc()
    
    print("\n" + "="*70)
    print("FINAL TEST SUMMARY")
    print("="*70)
    print(f"Tests Passed: {passed}/{len(test_functions)}")
    print(f"Tests Failed: {failed}/{len(test_functions)}")
    
    if failed == 0:
        print("\n🎉 ALL TESTS PASSED SUCCESSFULLY! 🎉")
    else:
        print(f"\n⚠️  {failed} TEST(S) FAILED - REVIEW OUTPUT ABOVE ⚠️")
    
    return passed, failed


if __name__ == "__main__":
    passed, failed = run_comprehensive_tests()
    
    # Exit with appropriate code
    import sys
    sys.exit(0 if failed == 0 else 1)


COMPREHENSIVE TEST SUITE FOR QuantumManagerStabilizer

TEST: Complex Scenarios

1. Quantum teleportation scenario:
   Alice's data qubit prepared
   Bell pair created between Alice and Bob
   Alice's operations complete
   All three qubits grouped? True
   ✓ Teleportation circuit prepared correctly

2. Three-qubit repetition code:
   Encoded |+⟩ in repetition code
   All qubits grouped? True
   Circuit: H 0
CX 0 1 0 2
   Added X error on middle qubit
   Circuit after error: H 0
CX 0 1 0 2
X 1
3
   ✓ Error correction code simulation successful

3. Parallel operations on separate qubit groups:
   Group 1 (Bell): keys [3, 4]
   Group 2 (GHZ): keys [5, 6, 7]
   Groups separate? True
   ✓ Parallel quantum operations maintained separately

✓✓✓ test_complex_scenarios PASSED ✓✓✓

FINAL TEST SUMMARY
Tests Passed: 1/1
Tests Failed: 0/1

🎉 ALL TESTS PASSED SUCCESSFULLY! 🎉


SystemExit: 0

## Function to convert amplitudes to stim.Circuit()

In [11]:
def amplitudes_to_stim_circuit(amplitudes):
    """
    Convert amplitude array to Stim circuit that prepares that state from |0...0⟩.
    
    Args:
        amplitudes: List of complex numbers representing quantum state amplitudes
    
    Returns:
        stim.Circuit that prepares the state (using indices 0, 1, 2...)
        
    Raises:
        ValueError: If amplitudes invalid or state not stabilizer
    """
    import numpy as np
    import stim
    
    amplitudes = np.array(amplitudes, dtype=complex)
    n = len(amplitudes)
    
    # Validate input
    if n == 0 or (n & (n - 1)) != 0:
        raise ValueError(f"Amplitude array length {n} must be a power of 2")
    
    num_qubits = int(np.log2(n))
    
    # Check normalization
    norm = np.sum(np.abs(amplitudes)**2)
    if not np.isclose(norm, 1.0, rtol=1e-10):
        raise ValueError(f"Amplitudes not normalized: norm = {norm}")
    
    circuit = stim.Circuit()
    
    # Single qubit states
    if num_qubits == 1:
        if np.allclose(amplitudes, [1, 0]):
            return circuit  # |0⟩
        elif np.allclose(amplitudes, [0, 1]):
            circuit.append("X", [0])  # |1⟩
        elif np.allclose(amplitudes, [1/np.sqrt(2), 1/np.sqrt(2)]):
            circuit.append("H", [0])  # |+⟩
        elif np.allclose(amplitudes, [1/np.sqrt(2), -1/np.sqrt(2)]):
            circuit.append("H", [0])
            circuit.append("Z", [0])  # |-⟩
        elif np.allclose(amplitudes, [1/np.sqrt(2), 1j/np.sqrt(2)]):
            circuit.append("H", [0])
            circuit.append("S", [0])  # |i⟩
        elif np.allclose(amplitudes, [1/np.sqrt(2), -1j/np.sqrt(2)]):
            circuit.append("H", [0])
            circuit.append("S_DAG", [0])  # |-i⟩
        else:
            raise ValueError(f"Single-qubit state is not a stabilizer state")
        return circuit
    
    # Multi-qubit computational basis states
    nonzero = np.where(np.abs(amplitudes) > 1e-10)[0]
    if len(nonzero) == 1:
        idx = nonzero[0]
        if not np.isclose(amplitudes[idx], 1.0):
            raise ValueError(f"Non-unit amplitude for basis state")
        # Apply X gates for each 1 bit
        for q in range(num_qubits):
            if idx & (1 << q):
                circuit.append("X", [q])
        return circuit
    
    # Two qubit states
    if num_qubits == 2:
        # Bell states
        if np.allclose(amplitudes, [1/np.sqrt(2), 0, 0, 1/np.sqrt(2)]):
            # |Φ+⟩
            circuit.append("H", [0])
            circuit.append("CX", [0, 1])
        elif np.allclose(amplitudes, [1/np.sqrt(2), 0, 0, -1/np.sqrt(2)]):
            # |Φ-⟩
            circuit.append("H", [0])
            circuit.append("CX", [0, 1])
            circuit.append("Z", [0])
        elif np.allclose(amplitudes, [0, 1/np.sqrt(2), 1/np.sqrt(2), 0]):
            # |Ψ+⟩
            circuit.append("H", [0])
            circuit.append("CX", [0, 1])
            circuit.append("X", [0])
        elif np.allclose(amplitudes, [0, 1/np.sqrt(2), -1/np.sqrt(2), 0]):
            # |Ψ-⟩
            circuit.append("H", [0])
            circuit.append("CX", [0, 1])
            circuit.append("X", [0])
            circuit.append("Z", [0])
        # Product states
        elif np.allclose(amplitudes, [0.5, 0.5, 0.5, 0.5]):
            # |++⟩
            circuit.append("H", [0])
            circuit.append("H", [1])
        elif np.allclose(amplitudes, [0.5, -0.5, 0.5, -0.5]):
            # |+-⟩
            circuit.append("H", [0])
            circuit.append("H", [1])
            circuit.append("Z", [1])
        else:
            raise ValueError("2-qubit state is not a recognized stabilizer state")
        return circuit
    
    # Three qubit GHZ state
    if num_qubits == 3:
        ghz = np.zeros(8)
        ghz[0] = ghz[7] = 1/np.sqrt(2)
        if np.allclose(amplitudes, ghz):
            circuit.append("H", [0])
            circuit.append("CX", [0, 1])
            circuit.append("CX", [0, 2])
            return circuit
    
    raise ValueError(f"{num_qubits}-qubit state is not a recognized stabilizer state")

#### Test amplitudes to stim

In [18]:
def test_amplitudes_to_stim_circuit():
    """Corrected tests based on actual Stim output."""
    import numpy as np
    import stim
    
    print("Testing amplitudes_to_stim_circuit function")
    print("=" * 50)
    
    # Test 1: Single qubit states
    print("\n1. SINGLE QUBIT STATES:")
    
    test_cases_1q = [
        ([1, 0], "", "|0⟩"),
        ([0, 1], "X 0", "|1⟩"),
        ([1/np.sqrt(2), 1/np.sqrt(2)], "H 0", "|+⟩"),
        ([1/np.sqrt(2), -1/np.sqrt(2)], "H 0\nZ 0", "|-⟩"),
        ([1/np.sqrt(2), 1j/np.sqrt(2)], "H 0\nS 0", "|i⟩"),
        ([1/np.sqrt(2), -1j/np.sqrt(2)], "H 0\nS_DAG 0", "|-i⟩"),
    ]
    
    for amps, expected, name in test_cases_1q:
        circuit = amplitudes_to_stim_circuit(amps)
        actual = str(circuit)
        assert actual == expected, f"Failed {name}: expected '{expected}', got '{actual}'"
        print(f"   ✓ {name} → {actual if actual else 'empty'}")
    
    # Test 2: Two qubit computational basis states
    print("\n2. TWO QUBIT COMPUTATIONAL BASIS:")
    
    test_cases_2q_comp = [
        ([1, 0, 0, 0], "", "|00⟩"),
        ([0, 1, 0, 0], "X 0", "|01⟩"),
        ([0, 0, 1, 0], "X 1", "|10⟩"),
        ([0, 0, 0, 1], "X 0 1", "|11⟩"),  # Note: Stim combines X gates
    ]
    
    for amps, expected, name in test_cases_2q_comp:
        circuit = amplitudes_to_stim_circuit(amps)
        actual = str(circuit)
        assert actual == expected, f"Failed {name}: expected '{expected}', got '{actual}'"
        print(f"   ✓ {name} → {actual if actual else 'empty'}")
    
    # Test 3: Bell states
    print("\n3. BELL STATES:")
    
    test_cases_bell = [
        ([1/np.sqrt(2), 0, 0, 1/np.sqrt(2)], "H 0\nCX 0 1", "|Φ+⟩"),
        ([1/np.sqrt(2), 0, 0, -1/np.sqrt(2)], "H 0\nCX 0 1\nZ 0", "|Φ-⟩"),
        ([0, 1/np.sqrt(2), 1/np.sqrt(2), 0], "H 0\nCX 0 1\nX 0", "|Ψ+⟩"),
        ([0, 1/np.sqrt(2), -1/np.sqrt(2), 0], "H 0\nCX 0 1\nX 0\nZ 0", "|Ψ-⟩"),
    ]
    
    for amps, expected, name in test_cases_bell:
        circuit = amplitudes_to_stim_circuit(amps)
        actual = str(circuit)
        assert actual == expected, f"Failed {name}: expected '{expected}', got '{actual}'"
        print(f"   ✓ {name}")
    
    # Test 4: Product states
    print("\n4. PRODUCT STATES:")
    
    test_cases_product = [
        ([0.5, 0.5, 0.5, 0.5], "H 0 1", "|++⟩"),  # Stim combines H gates
        ([0.5, -0.5, 0.5, -0.5], "H 0 1\nZ 1", "|+-⟩"),
    ]
    
    for amps, expected, name in test_cases_product:
        circuit = amplitudes_to_stim_circuit(amps)
        actual = str(circuit)
        assert actual == expected, f"Failed {name}: expected '{expected}', got '{actual}'"
        print(f"   ✓ {name}")
    
    # Test 5: Three qubit states
    print("\n5. THREE QUBIT STATES:")
    
    # Specific cases
    test_cases_3q = [
        ([1, 0, 0, 0, 0, 0, 0, 0], "", "|000⟩"),
        ([0, 1, 0, 0, 0, 0, 0, 0], "X 0", "|001⟩"),
        ([0, 0, 0, 0, 0, 1, 0, 0], "X 0 2", "|101⟩"),  # Stim combines X gates
        ([0, 0, 0, 0, 0, 0, 0, 1], "X 0 1 2", "|111⟩"),  # All three X gates combined
    ]
    
    for amps, expected, name in test_cases_3q:
        circuit = amplitudes_to_stim_circuit(amps)
        actual = str(circuit)
        assert actual == expected, f"Failed {name}: expected '{expected}', got '{actual}'"
        print(f"   ✓ {name}")
    
    # GHZ state
    ghz = [0] * 8
    ghz[0] = 1/np.sqrt(2)
    ghz[7] = 1/np.sqrt(2)
    circuit = amplitudes_to_stim_circuit(ghz)
    actual = str(circuit)
    # Note: CX gates might be combined too
    assert "H 0" in actual and "CX" in actual
    print(f"   ✓ GHZ state")
    
    # Test 6: Error cases
    print("\n6. ERROR HANDLING:")
    
    error_cases = [
        ([1, 1], "not normalized", "Non-normalized"),
        ([1, 0, 0], "power of 2", "Wrong length"),
        ([], "0", "Empty list"),
        ([np.sqrt(0.3), np.sqrt(0.7)], "not a stabilizer", "Non-stabilizer"),
    ]
    
    for amps, expected_error, name in error_cases:
        try:
            circuit = amplitudes_to_stim_circuit(amps)
            assert False, f"{name} should have raised ValueError"
        except ValueError as e:
            assert expected_error in str(e).lower()
            print(f"   ✓ {name} → Error caught")
    
    print("\n" + "=" * 50)
    print("All tests passed! ✅")

if __name__ == "__main__":
    test_amplitudes_to_stim_circuit()

Testing amplitudes_to_stim_circuit function

1. SINGLE QUBIT STATES:
   ✓ |0⟩ → empty
   ✓ |1⟩ → X 0
   ✓ |+⟩ → H 0
   ✓ |-⟩ → H 0
Z 0
   ✓ |i⟩ → H 0
S 0
   ✓ |-i⟩ → H 0
S_DAG 0

2. TWO QUBIT COMPUTATIONAL BASIS:
   ✓ |00⟩ → empty
   ✓ |01⟩ → X 0
   ✓ |10⟩ → X 1
   ✓ |11⟩ → X 0 1

3. BELL STATES:
   ✓ |Φ+⟩
   ✓ |Φ-⟩
   ✓ |Ψ+⟩
   ✓ |Ψ-⟩

4. PRODUCT STATES:
   ✓ |++⟩
   ✓ |+-⟩

5. THREE QUBIT STATES:
   ✓ |000⟩
   ✓ |001⟩
   ✓ |101⟩
   ✓ |111⟩
   ✓ GHZ state

6. ERROR HANDLING:
   ✓ Non-normalized → Error caught
   ✓ Wrong length → Error caught
   ✓ Empty list → Error caught
   ✓ Non-stabilizer → Error caught

All tests passed! ✅
